We want to base on our past movie data to recommend similar movies based on similar plot, genre, director, etc. For example if we search for Titanic movie, similar movies should be recommended to us, which can be done based on the plots, directors, cast, etc.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer ## Same function as Document Term Frequency
pd.set_option('display.max_columns', 100)
df=pd.read_csv("https://query.data.world/s/uikepcpffyo2nhig52xxeevdialfl7")
df.head()

,Unnamed: 0,Title,Year,Rated,Released,Runtime,Genre,Director,Writer,Actors,Plot,Language,Country,Awards,Poster,Ratings.Source,Ratings.Value,Metascore,imdbRating,imdbVotes,imdbID,Type,tomatoMeter,tomatoImage,tomatoRating,tomatoReviews,tomatoFresh,tomatoRotten,tomatoConsensus,tomatoUserMeter,tomatoUserRating,tomatoUserReviews,tomatoURL,DVD,BoxOffice,Production,Website,Response
0,1,The Shawshank Redemption,1994,R,14 Oct 1994,142 min,"Crime, Drama",Frank Darabont,"Stephen King (short story ""Rita Hayworth and S...","Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,English,USA,Nominated for 7 Oscars. Another 19 wins & 30 n...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.3/10,80.0,9.3,"1,825,626",tt0111161,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/shawshank_rede...,27 Jan 1998,NaN,Columbia Pictures,NaN,True
1,2,The Godfather,1972,R,24 Mar 1972,175 min,"Crime, Drama",Francis Ford Coppola,"Mario Puzo (screenplay), Francis Ford Coppola ...","Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,"English, Italian, Latin",USA,Won 3 Oscars. Another 23 wins & 27 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.2/10,100.0,9.2,"1,243,444",tt0068646,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather/,09 Oct 2001,NaN,Paramount Pictures,http://www.thegodfather.com,True
2,3,The Godfather: Part II,1974,R,20 Dec 1974,202 min,"Crime, Drama",Francis Ford Coppola,"Francis Ford Coppola (screenplay), Mario Puzo ...","Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,"English, Italian, Spanish, Latin, Sicilian",USA,Won 6 Oscars. Another 10 wins & 20 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,85.0,9.0,"856,870",tt0071562,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/godfather_part...,24 May 2005,NaN,Paramount Pictures,http://www.thegodfather.com/,True
3,4,The Dark Knight,2008,PG-13,18 Jul 2008,152 min,"Action, Crime, Drama",Christopher Nolan,"Jonathan Nolan (screenplay), Christopher Nolan...","Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...,"English, Mandarin","USA, UK",Won 2 Oscars. Another 151 wins & 153 nominations.,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,9.0/10,82.0,9.0,"1,802,351",tt0468569,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/the_dark_knight/,09 Dec 2008,"$533,316,061",Warner Bros. Pictures/Legendary,http://thedarkknight.warnerbros.com/,True
4,5,12 Angry Men,1957,APPROVED,01 Apr 1957,96 min,"Crime, Drama",Sidney Lumet,"Reginald Rose (story), Reginald Rose (screenplay)","Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...,English,USA,Nominated for 3 Oscars. Another 16 wins & 8 no...,https://images-na.ssl-images-amazon.com/images...,Internet Movie Database,8.9/10,96.0,8.9,"494,215",tt0050083,movie,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.rottentomatoes.com/m/1000013-12_ang...,06 Mar 2001,NaN,Criterion Collection,http://www.criterion.com/films/27871-12-angry-men,True


In [ ]:
df.shape

(250, 38)

## We will base on "Title","Genre","Director","Actors","Plot" to make recommendations

In [ ]:
df=df[["Title","Genre","Director","Actors","Plot"]]
df.head()

,Title,Genre,Director,Actors,Plot
0,The Shawshank Redemption,"Crime, Drama",Frank Darabont,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...
1,The Godfather,"Crime, Drama",Francis Ford Coppola,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...
2,The Godfather: Part II,"Crime, Drama",Francis Ford Coppola,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...
3,The Dark Knight,"Action, Crime, Drama",Christopher Nolan,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker emerges fro...
4,12 Angry Men,"Crime, Drama",Sidney Lumet,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",A jury holdout attempts to prevent a miscarria...


In [ ]:
## Discarding commas between actors names
df['Actors']=df['Actors'].map(lambda x: x.lower().split(","))

In [ ]:
df['Actors']

0      [tim robbins,  morgan freeman,  bob gunton,  w...
1      [marlon brando,  al pacino,  james caan,  rich...
2      [al pacino,  robert duvall,  diane keaton,  ro...
3      [christian bale,  heath ledger,  aaron eckhart...
4      [martin balsam,  john fiedler,  lee j. cobb,  ...
                             ...                        
245    [ray milland,  jane wyman,  phillip terry,  ho...
246    [brie larson,  john gallagher jr.,  stephanie ...
247    [cary grant,  rosalind russell,  ralph bellamy...
248    [sissy spacek,  jane galloway heitz,  joseph a...
249    [dev patel,  saurabh shukla,  anil kapoor,  ra...
Name: Actors, Length: 250, dtype: object

In [ ]:
## Putting genres in a list of words
df['Genre']=df["Genre"].map(lambda x: x.lower().split(","))

In [ ]:
df['Genre']

0                 [crime,  drama]
1                 [crime,  drama]
2                 [crime,  drama]
3        [action,  crime,  drama]
4                 [crime,  drama]
                  ...            
245           [drama,  film-noir]
246                       [drama]
247    [comedy,  drama,  romance]
248           [biography,  drama]
249                       [drama]
Name: Genre, Length: 250, dtype: object

In [ ]:
df['Director']=df['Director'].map(lambda x: x.lower().split(" "))

In [ ]:
df['Director']

0                      [frank, darabont]
1               [francis, ford, coppola]
2               [francis, ford, coppola]
3                   [christopher, nolan]
4                        [sidney, lumet]
                     ...                
245                      [billy, wilder]
246            [destin, daniel, cretton]
247                      [howard, hawks]
248                       [david, lynch]
249    [danny, boyle,, loveleen, tandan]
Name: Director, Length: 250, dtype: object

## Convert names to lower cases and avoid duplicates

In [ ]:
for index,row in df.iterrows():
  row['Actors']=[x.lower().replace(' ','') for x in row['Actors']]
  row['Director']=''.join(row['Director']).lower()

In [ ]:
df['Actors']

0      [timrobbins, morganfreeman, bobgunton, william...
1      [marlonbrando, alpacino, jamescaan, richards.c...
2      [alpacino, robertduvall, dianekeaton, robertde...
3      [christianbale, heathledger, aaroneckhart, mic...
4      [martinbalsam, johnfiedler, leej.cobb, e.g.mar...
                             ...                        
245    [raymilland, janewyman, phillipterry, howardda...
246    [brielarson, johngallagherjr., stephaniebeatri...
247    [carygrant, rosalindrussell, ralphbellamy, gen...
248    [sissyspacek, janegallowayheitz, josepha.carpe...
249     [devpatel, saurabhshukla, anilkapoor, rajzutshi]
Name: Actors, Length: 250, dtype: object

In [ ]:
df['Director']

0                  frankdarabont
1             francisfordcoppola
2             francisfordcoppola
3               christophernolan
4                    sidneylumet
                 ...            
245                  billywilder
246          destindanielcretton
247                  howardhawks
248                   davidlynch
249    dannyboyle,loveleentandan
Name: Director, Length: 250, dtype: object

In [ ]:
!pip install rake_nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import rake_nltk
from rake_nltk import Rake ## For extracting significant keywords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
## Initiaizing the new columns
df['Key_words']=""
for index,row in df.iterrows():
  plot=row['Plot']

## Instantiating rake and discard all punctuation marks
  r=Rake()

## Getting the dictionary with key words and their score
  r.extract_keywords_from_text(plot)

  key_words_dict_scores = r.get_word_degrees()

# assigning the key words to the new column
  row['Key_words'] = list(key_words_dict_scores.keys())

df.drop(columns = ['Plot'], inplace = True)

In [ ]:
key_words_dict_scores

defaultdict(<function rake_nltk.rake.Rake._build_word_co_occurance_graph.<locals>.<lambda>()>,
            {'mumbai': 3,
             'teen': 3,
             'reflects': 3,
             'upbringing': 1,
             'slums': 1,
             'accused': 1,
             'cheating': 1,
             'indian': 2,
             'version': 2,
             'wants': 1,
             'millionaire': 2,
             '?"': 2})

In [ ]:
df

,Title,Genre,Director,Actors,Key_words
0,The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton, william...","[two, imprisoned, men, bond, number, years, fi..."
1,The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan, richards.c...","[aging, patriarch, organized, crime, dynasty, ..."
2,The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton, robertde...","[early, life, career, vito, corleone, 1920s, n..."
3,The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart, mic...","[menace, known, joker, emerges, mysterious, pa..."
4,12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb, e.g.mar...","[jury, holdout, attempts, prevent, miscarriage..."
...,...,...,...,...,...
245,The Lost Weekend,"[drama, film-noir]",billywilder,"[raymilland, janewyman, phillipterry, howardda...","[desperate, life, chronic, alcoholic, followed..."
246,Short Term 12,[drama],destindanielcretton,"[brielarson, johngallagherjr., stephaniebeatri...","[20, something, supervising, staff, member, re..."
247,His Girl Friday,"[comedy, drama, romance]",howardhawks,"[carygrant, rosalindrussell, ralphbellamy, gen...","[newspaper, editor, uses, every, trick, book, ..."
248,The Straight Story,"[biography, drama]",davidlynch,"[sissyspacek, janegallowayheitz, josepha.carpe...","[old, man, makes, long, journey, lawn, mover, ..."


In [ ]:
df.set_index("Title",inplace=True)
df

,Genre,Director,Actors,Key_words
Title,,,,
The Shawshank Redemption,"[crime, drama]",frankdarabont,"[timrobbins, morganfreeman, bobgunton, william...","[two, imprisoned, men, bond, number, years, fi..."
The Godfather,"[crime, drama]",francisfordcoppola,"[marlonbrando, alpacino, jamescaan, richards.c...","[aging, patriarch, organized, crime, dynasty, ..."
The Godfather: Part II,"[crime, drama]",francisfordcoppola,"[alpacino, robertduvall, dianekeaton, robertde...","[early, life, career, vito, corleone, 1920s, n..."
The Dark Knight,"[action, crime, drama]",christophernolan,"[christianbale, heathledger, aaroneckhart, mic...","[menace, known, joker, emerges, mysterious, pa..."
12 Angry Men,"[crime, drama]",sidneylumet,"[martinbalsam, johnfiedler, leej.cobb, e.g.mar...","[jury, holdout, attempts, prevent, miscarriage..."
...,...,...,...,...
The Lost Weekend,"[drama, film-noir]",billywilder,"[raymilland, janewyman, phillipterry, howardda...","[desperate, life, chronic, alcoholic, followed..."
Short Term 12,[drama],destindanielcretton,"[brielarson, johngallagherjr., stephaniebeatri...","[20, something, supervising, staff, member, re..."
His Girl Friday,"[comedy, drama, romance]",howardhawks,"[carygrant, rosalindrussell, ralphbellamy, gen...","[newspaper, editor, uses, every, trick, book, ..."


## Bag of Words

In [ ]:
df['bag_of_words']=''
columns=df.columns
for index,row in df.iterrows():
  words=''
  for col in columns:
    if col!='Director':
      words=words+" ".join(row[col])+" "
    else:
      words=words+row[col]+' '
    row['bag_of_words']=words

df.drop(columns=[col for col in df.columns if col!='bag_of_words'], inplace=True)

In [ ]:
df

,bag_of_words
Title,
The Shawshank Redemption,crime drama frankdarabont timrobbins morganfr...
The Godfather,crime drama francisfordcoppola marlonbrando a...
The Godfather: Part II,crime drama francisfordcoppola alpacino rober...
The Dark Knight,action crime drama christophernolan christia...
12 Angry Men,crime drama sidneylumet martinbalsam johnfied...
...,...
The Lost Weekend,drama film-noir billywilder raymilland janewy...
Short Term 12,drama destindanielcretton brielarson johngalla...
His Girl Friday,comedy drama romance howardhawks carygrant r...


In [ ]:
## Generating Count Vectorizer
count=CountVectorizer()
count_matrix=count.fit_transform(df['bag_of_words'])

In [ ]:
count_matrix

<250x3176 sparse matrix of type '<class 'numpy.int64'>'
	with 5601 stored elements in Compressed Sparse Row format>

In [ ]:
c=count_matrix.todense()

In [ ]:
c

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
print(count_matrix[0,:])

  (0, 619)	1
  (0, 816)	1
  (0, 1073)	1
  (0, 2877)	1
  (0, 1962)	1
  (0, 323)	1
  (0, 3116)	1
  (0, 2966)	1
  (0, 1346)	1
  (0, 1879)	1
  (0, 328)	1
  (0, 2055)	1
  (0, 3164)	1
  (0, 1028)	1
  (0, 2672)	1
  (0, 947)	1
  (0, 2344)	1
  (0, 59)	1
  (0, 553)	1
  (0, 700)	1


In [ ]:
## Generating the cosine similarity matrix
cosine_sim=cosine_similarity(count_matrix,count_matrix)
cosine_sim

array([[1.        , 0.14638501, 0.1315587 , ..., 0.05      , 0.05      ,
        0.05270463],
       [0.14638501, 1.        , 0.34236839, ..., 0.048795  , 0.048795  ,
        0.05143445],
       [0.1315587 , 0.34236839, 1.        , ..., 0.0438529 , 0.0438529 ,
        0.04622502],
       ...,
       [0.05      , 0.048795  , 0.0438529 , ..., 1.        , 0.05      ,
        0.05270463],
       [0.05      , 0.048795  , 0.0438529 , ..., 0.05      , 1.        ,
        0.05270463],
       [0.05270463, 0.05143445, 0.04622502, ..., 0.05270463, 0.05270463,
        1.        ]])

In [ ]:
# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes

indices=pd.Series(df.index)
indices[:20]

0                              The Shawshank Redemption
1                                         The Godfather
2                                The Godfather: Part II
3                                       The Dark Knight
4                                          12 Angry Men
5                                      Schindler's List
6         The Lord of the Rings: The Return of the King
7                                          Pulp Fiction
8                                            Fight Club
9     The Lord of the Rings: The Fellowship of the Ring
10                                         Forrest Gump
11       Star Wars: Episode V - The Empire Strikes Back
12                                            Inception
13                The Lord of the Rings: The Two Towers
14                      One Flew Over the Cuckoo's Nest
15                                           Goodfellas
16                                           The Matrix
17                   Star Wars: Episode IV - A N

In [ ]:
## Creating a Function that takes movie title as the input and returns top 10 recommended movies
def recommendations(title, coine_sim=cosine_sim):
  recommended_movies=[]
  ## Getting the index of the movie that matches the title
  idx=indices[indices==title].index[0]
  ## Creating a Series with the similarity scores in descending order
  score_series=pd.Series(cosine_sim[idx]).sort_values(ascending=False)
  ## Getting the indexes of 10 most similar movies
  top_10_indexes=list(score_series.iloc[1:11].index)
  print(top_10_indexes)
  ## Populating the list with the titles of the best 10 matching movies
  for i in top_10_indexes:
    recommended_movies.append(list(df.index)[i])
  return recommended_movies

In [ ]:
recommendations('The Godfather')

[2, 83, 128, 226, 100, 15, 76, 123, 110, 66]


['The Godfather: Part II',
 'Scarface',
 'Fargo',
 'Rope',
 'On the Waterfront',
 'Goodfellas',
 'Baby Driver',
 'Cool Hand Luke',
 'Casino',
 'A Clockwork Orange']

In [ ]:
recommendations('Fargo')

[132, 34, 226, 125, 61, 1, 2, 100, 15, 214]


['No Country for Old Men',
 'The Departed',
 'Rope',
 'The Big Lebowski',
 'Reservoir Dogs',
 'The Godfather',
 'The Godfather: Part II',
 'On the Waterfront',
 'Goodfellas',
 'Arsenic and Old Lace']